Наиболее известная и популярная упрощенная модель – это искусственный нейрон Маккалока — Питтса, разработанный в начале 1940-х годов 

In [ ]:
using Pkg
pkgs = ["Plots", "TextParse", "CSV", "DataFrames", "ImageMagick", "Images", "Interact", "Flux"]

for p in pkgs
    Pkg.add(p)
end

for p in pkgs
    Pkg.build(p)
end

In [ ]:
using Images

fnames = [
    "data/10_100.jpg",
    "data/107_100.jpg",
    "data/yellow_apple_2.jpg",
    "data/8_100.jpg",
    "data/104_100.jpg", 
    "data/3_100.jpg"
]
# массив картинок
fruits = [load(fname) for fname in fnames]

hcat(fruits...) # соединить в одно изображение

In [ ]:
using Statistics: mean

M1 = [ mean(float.(c.(img))) for c = [red,green,blue], img = fruits ]

In [ ]:
function tweaking(img)
    R = colorview( RGB, red.(img),zeroarray,zeroarray )
    G = colorview( RGB, zeroarray,green.(img),zeroarray )
    B = colorview( RGB, zeroarray,zeroarray, blue.(img) )
    [R; G; B]
end

tweaking( hcat(fruits...) )

In [ ]:
function negative(img)
    Arr = channelview(img)
    Brr = ones(size(Arr)) - Arr
    colorview(RGB, Brr)
end

antifruits = negative.(fruits)
hcat(antifruits...)

In [ ]:
tweaking( hcat(antifruits...) )

In [ ]:
M2 = [ mean(float.(c.(img))) for c = [red,green,blue], img = antifruits ]

In [ ]:
function remove_background(img)
    mtrx = copy( channelview(img) )
    for i = 1:size(mtrx, 2), j = 1:size(mtrx, 3)
        if reduce(&, mtrx[:,i,j] .> [0.8, 0.8, 0.8])
            mtrx[:,i,j] .= [0.5, 0.5, 0.5]
        end
    end
    colorview(RGB, mtrx)
end

greyfruits = remove_background.(fruits) # боги смерти едят только яблоки

In [ ]:
M3 = [ mean(float.(c.(img))) for c = [red,green,blue], img = greyfruits ]

https://github.com/Horea94/Fruit-Images-Dataset

In [ ]:
using TextParse

In [ ]:
applecols, applecolnames = TextParse.csvread("data/Apple_Braeburn.dat", '\t')
bananacols, bananacolnames = TextParse.csvread("data/bananas.dat", '\t')

In [ ]:
using DataFrames, CSV

In [ ]:
banans = DataFrame([col for col in bananacols], [Meta.parse(bc) for bc in bananacolnames])
apples = DataFrame([col for col in applecols], [Meta.parse(ac) for ac in applecolnames])

In [ ]:
pth = "C:\\Users\\User\\Desktop\\Banana" # Apple Braeburn
fnames = readdir(pth)[1:300]
#load("$pth\\320_100.jpg")

In [ ]:
dataz = []
for fname in fnames
    img_i = load("$pth\\$fname")
    gbimg = remove_background(img_i)
    colorz = [ mean(c.(gbimg)) for c = [red,green,blue] ]
    inform = [size(gbimg, 1) size(gbimg, 2) colorz' ]
    
    push!(dataz, inform)
end
dataz

In [ ]:
banans = DataFrame( vcat(dataz...), [:height, :width, :red, :green, :blue] )

In [ ]:
apples = DataFrame( vcat(dataz...), [:height, :width, :red, :green, :blue] )

In [ ]:
CSV.write("data/Apple_Braeburn.csv", apples)
CSV.write("data/bananas.csv", banans)